In [1]:
# -*- coding: utf-8 -*-

import config
import pandas as pd
import math
import numpy as np
from datetime import datetime
from tools import datacleaner as dtclean
import matplotlib.pyplot as plt

In [2]:
def prep_new_dataset(
        store=None,
        variable=None,
        full_dataframe=False,
        sheet_name='Base',
        filename='/data/Venda_274_432.xlsx',
        cols=None):
    path = getattr(config,'path','default_value')
    cli_data_file = path+filename
    raw_data =pd.read_excel(cli_data_file,
                            sheet_name=sheet_name,
                            header=0,
                            index_col=None,
                            parse_dates={'period':[1]},
                            keep_default_na=True
                            )
    pd.to_datetime(raw_data['period'])
    #raw_data.astype({'Dia Semana': 'str'}).dtypes
    return raw_data

In [3]:
df= prep_new_dataset(sheet_name=5,filename='/data/teste_new_ds.xlsx')
df.head()

,period,Store,hora,Hour,Minute,ValuePLN,Units,Clients,TimeStamp
0,2018-01-02,1027,8:30,8,30,96.019997,34,10,2020-08-19 07:34:21.880000000
1,2018-01-02,1027,9:00,9,0,373.140010,129,28,2020-08-19 07:34:21.880000000
2,2018-01-02,1027,9:30,9,30,449.030000,113,18,2020-08-19 07:34:21.880000000
3,2018-01-02,1027,10:0,10,0,194.480000,42,14,2020-08-19 07:34:21.880000000
4,2018-01-02,1027,10:30,10,30,323.579990,83,15,2020-08-19 07:34:21.880000000


In [4]:
df = dtclean.convert_date_time_values(df)
df.head()

,period,Store,hora,Hour,Minute,ValuePLN,Units,Clients,TimeStamp,ds
0,2018-01-02,1027,8:30,8,30,96.019997,34,10,2020-08-19 07:34:21.880000000,2018-01-02 08:30:00
1,2018-01-02,1027,9:00,9,0,373.140010,129,28,2020-08-19 07:34:21.880000000,2018-01-02 09:00:00
2,2018-01-02,1027,9:30,9,30,449.030000,113,18,2020-08-19 07:34:21.880000000,2018-01-02 09:30:00
3,2018-01-02,1027,10:0,10,0,194.480000,42,14,2020-08-19 07:34:21.880000000,2018-01-02 10:00:00
4,2018-01-02,1027,10:30,10,30,323.579990,83,15,2020-08-19 07:34:21.880000000,2018-01-02 10:30:00


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170773 entries, 0 to 170772
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   period     170773 non-null  datetime64[ns]
 1   Store      170773 non-null  int64         
 2   hora       7 non-null       object        
 3   Hour       170773 non-null  int64         
 4   Minute     170773 non-null  int64         
 5   ValuePLN   170773 non-null  float64       
 6   Units      170773 non-null  int64         
 7   Clients    170773 non-null  int64         
 8   TimeStamp  170773 non-null  object        
 9   ds         170773 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(5), object(2)
memory usage: 13.0+ MB


In [7]:
print('Data início: {}, Data final: {}'.format(df.ds.min(),df.ds.max()))
df.info()


Data início: 2018-01-02 08:00:00, Data final: 2020-08-26 21:00:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170773 entries, 0 to 170772
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   period     170773 non-null  datetime64[ns]
 1   Store      170773 non-null  int64         
 2   hora       7 non-null       object        
 3   Hour       170773 non-null  int64         
 4   Minute     170773 non-null  int64         
 5   ValuePLN   170773 non-null  float64       
 6   Units      170773 non-null  int64         
 7   Clients    170773 non-null  int64         
 8   TimeStamp  170773 non-null  object        
 9   ds         170773 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(5), object(2)
memory usage: 13.0+ MB


In [14]:
df.drop(columns=['TimeStamp','Hour','Minute'],inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170773 entries, 0 to 170772
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   period    170773 non-null  datetime64[ns]
 1   Store     170773 non-null  int64         
 2   ValuePLN  170773 non-null  float64       
 3   Units     170773 non-null  int64         
 4   Clients   170773 non-null  int64         
 5   ds        170773 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(3)
memory usage: 7.8 MB


In [15]:
df_grouped_store = df.groupby('Store')
df_grouped_store.head()

,period,Store,ValuePLN,Units,Clients,ds
0,2018-01-02,1027,96.019997,34,10,2018-01-02 08:30:00
1,2018-01-02,1027,373.140010,129,28,2018-01-02 09:00:00
2,2018-01-02,1027,449.030000,113,18,2018-01-02 09:30:00
3,2018-01-02,1027,194.480000,42,14,2018-01-02 10:00:00
4,2018-01-02,1027,323.579990,83,15,2018-01-02 10:30:00
20482,2018-01-03,259,-5.430000,0,-1,2018-01-03 16:30:00
20483,2018-01-05,259,-1.970000,0,-1,2018-01-05 11:30:00
20484,2018-01-11,259,-4.950000,0,-1,2018-01-11 19:00:00
20485,2018-01-12,259,-2.150000,0,-1,2018-01-12 12:00:00
20486,2018-01-12,259,-6.680000,0,-1,2018-01-12 18:00:00


In [ ]:
df[df['Units']<0]

In [ ]:
df[df['Clients']<0]

In [ ]:
mascara=(df['period']=='2020-04-02') & (df['Store']==61)
df.loc[mascara]

In [ ]:
df.loc[mascara,'Units'].sum()

,period,Store,ValuePLN,Units,Clients,ds
1722,2018-03-19,1027,-8.990000,-1,-1,2018-03-19 20:30:00
13493,2019-11-08,1027,-10.040000,-1,-2,2019-11-08 08:00:00
13762,2019-11-18,1027,-13.290000,-1,-1,2019-11-18 08:30:00
17849,2020-04-21,1027,-10.500000,-1,-1,2020-04-21 09:30:00
18090,2020-04-30,1027,-2.090000,-1,-1,2020-04-30 16:00:00
...,...,...,...,...,...,...
163924,2019-11-03,61,-13.800000,-12,-1,2019-11-03 19:30:00
164952,2019-12-13,61,-67.800003,-20,-1,2019-12-13 22:00:00
166803,2020-02-27,61,-19.459999,-4,-1,2020-02-27 22:00:00
167702,2020-04-02,61,-53.520000,-28,0,2020-04-02 22:00:00


In [17]:
df[df['Clients']<0]

,period,Store,ValuePLN,Units,Clients,ds
1722,2018-03-19,1027,-8.990000,-1,-1,2018-03-19 20:30:00
1929,2018-04-02,1027,-47.560001,0,-1,2018-04-02 13:30:00
1930,2018-04-10,1027,-2.110000,0,-1,2018-04-10 21:00:00
2411,2018-05-02,1027,73.709999,20,-1,2018-05-02 20:30:00
13493,2019-11-08,1027,-10.040000,-1,-2,2019-11-08 08:00:00
...,...,...,...,...,...,...
160411,2019-04-16,61,-86.620003,-18,-1,2019-04-16 07:00:00
160437,2019-04-16,61,-19.670000,-3,-1,2019-04-16 21:00:00
163924,2019-11-03,61,-13.800000,-12,-1,2019-11-03 19:30:00
164952,2019-12-13,61,-67.800003,-20,-1,2019-12-13 22:00:00


In [23]:
mascara=(df['period']=='2020-04-02') & (df['Store']==61)
df.loc[mascara]

,period,Store,ValuePLN,Units,Clients,ds
167675,2020-04-02,61,9.410000,3,1,2020-04-02 08:30:00
167676,2020-04-02,61,4.790000,4,2,2020-04-02 09:00:00
167677,2020-04-02,61,37.490002,11,5,2020-04-02 09:30:00
167678,2020-04-02,61,448.790010,91,16,2020-04-02 10:00:00
167679,2020-04-02,61,625.580020,123,14,2020-04-02 10:30:00
167680,2020-04-02,61,289.370000,70,9,2020-04-02 11:00:00
167681,2020-04-02,61,588.380000,146,23,2020-04-02 11:30:00
167682,2020-04-02,61,241.560000,89,10,2020-04-02 12:00:00
167683,2020-04-02,61,364.340000,91,15,2020-04-02 12:30:00
167684,2020-04-02,61,361.040010,94,19,2020-04-02 13:00:00


In [28]:
df.loc[mascara,'Units'].sum()

2871